# Вопрос 1

In [1]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_rows', 100)

# чтение данных из файла
df = pd.read_csv('company-test-task.csv') 
df.head()

,user_id,medium,event_name,page_location,transaction_id,to_timestamp
0,ea8ba0f60c61118f2718ec987f4b8edd,e414a106991585be104aec18f5820091,page_view,e174b8eefcfc6794d78994bb17a6cdb7,NaN,2022-09-01 14:20:54.143 +0300
1,e0664300f1aa81f4f8d50b228cdbde95,NaN,page_view,58cb72729bf998395d82200d0ed65851,NaN,2022-09-01 14:21:08.129 +0300
2,242ac3e4e3a30f9bc8db1b1c20c48ef4,219338c95a3f5d98e757bf98357345dd,page_view,cf1b5929aa7d67ebc4302c62f5099970,NaN,2022-09-01 14:21:38.044 +0300
3,f23ddc8568144988eb79ed638fa21b0e,NaN,page_view,220627fa9690338a0049dcc8d50c2b7f,NaN,2022-09-01 14:21:48.073 +0300
4,7561fceca77fb3dbe8d9ccc39620a583,3439029c3f5a75eeb3226ded1b0774d6,page_view,eb92be3a01c830d7c38376bcf3ce035b,NaN,2022-09-01 14:21:54.221 +0300


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58781 entries, 0 to 58780
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         58781 non-null  object
 1   medium          24934 non-null  object
 2   event_name      58781 non-null  object
 3   page_location   58781 non-null  object
 4   transaction_id  455 non-null    object
 5   to_timestamp    58781 non-null  object
dtypes: object(6)
memory usage: 2.7+ MB


In [3]:
# обнаружено 204 дублирующих строк
df.duplicated().sum() 

204

In [4]:
# удаляем дубликаты
df = df.drop_duplicates() 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58577 entries, 0 to 58780
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         58577 non-null  object
 1   medium          24933 non-null  object
 2   event_name      58577 non-null  object
 3   page_location   58577 non-null  object
 4   transaction_id  260 non-null    object
 5   to_timestamp    58577 non-null  object
dtypes: object(6)
memory usage: 3.1+ MB


In [5]:
print(df['medium'].unique())

['e414a106991585be104aec18f5820091' nan '219338c95a3f5d98e757bf98357345dd'
 '3439029c3f5a75eeb3226ded1b0774d6' '3d27728bd3edf2fea3fe3e44c66ceee4'
 '3b8294dc079c2882a8fe9b110c8da425' 'a52f6dfd8cad6dc0e93ad01a508da55f'
 '9dc53318c6e1ca7ffaabd439e3d631fc' 'ac642a883b4cce68519686112c9bed55'
 '9446aa3aeddbe34ab778380c8299c555' '797d47618ef55266adac1f57f51f42b9'
 '338bfbf125696d029d305ff3c3e2f16c' '969e356d90798959734b443270cc4982'
 '3ac790efaacb1de51d686ba120abef83' '8ed1dac3de2fedbe6531b3ef1937ded3'
 '6f95f4520491011658ec28246626b444' '027572bb7194cf20b350c352e71a5084'
 'e1cc3c5d68a34bea27cbca423967cae7' '29637e0d77fcd8d1d29c27e3ce91d251'
 'f6950f0c3bb3063f1384e10d8b5de273' '00cd8b365a6a45e64eb9bd48a938bd13'
 'de5027d17a4f2ac4d59f0fb9db5d358f' '22a1ba053c756f9129c8218e33d1369e'
 '901a475cefbde9f43a07e8ba444f9ac0' '15815864f725f2643728e2218db51eaf']


In [6]:
# рассчитываем время в минутах между 2-мя последовательными записями юзера
# если нет след-ей записи, то берем now() по Мск
df = df.sort_values(by=['user_id', 'to_timestamp'], ascending=True)  
df['to_timestamp_lead'] = df.groupby(['user_id'])['to_timestamp'].shift(-1) 
df['to_timestamp'] = pd.to_datetime(df['to_timestamp'])
df['to_timestamp_lead'] = df['to_timestamp_lead'].fillna(pd.Timestamp.utcnow())  
df['to_timestamp_lead'] = pd.to_datetime(df['to_timestamp_lead'], utc=True).dt.tz_convert(tz="Europe/Moscow")
df['min_diff'] = (df['to_timestamp_lead']-df['to_timestamp']).astype('timedelta64[m]') 
df.head()

,user_id,medium,event_name,page_location,transaction_id,to_timestamp,to_timestamp_lead,min_diff
1600,00023adaac7c3392684aee44f9744407,NaN,page_view,de9eb5f3106683e3aee36af54548f17b,NaN,2022-09-02 00:17:20.943000+03:00,2023-04-26 17:10:50.617251+03:00,340853.0
26530,000296445fbdf5da871d39fae90c32de,NaN,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-08 12:05:15.837000+03:00,2022-09-09 10:26:51.558000+03:00,1341.0
30806,000296445fbdf5da871d39fae90c32de,NaN,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-09 10:26:51.558000+03:00,2022-09-09 10:31:34.473000+03:00,4.0
30823,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,page_view,b5ee87931581ac101880ef3043eafd7d,NaN,2022-09-09 10:31:34.473000+03:00,2023-04-26 17:10:50.617251+03:00,330159.0
44535,0009db22643e3eb23f9098f341738546,NaN,page_view,2ccb2870dd72c5958634a326eee8e50b,NaN,2022-09-14 20:28:06.327000+03:00,2023-04-26 17:10:50.617251+03:00,322362.0


In [7]:
# добавляем колонку со следующим маркетинговым каналом юзера
df['medium'] = df['medium'].fillna('unknown')
df['medium_lead'] = df.groupby(['user_id'])['medium'].shift(-1) 
df['medium_lead'] = df['medium_lead'].fillna('unknown')
df.head()

,user_id,medium,event_name,page_location,transaction_id,to_timestamp,to_timestamp_lead,min_diff,medium_lead
1600,00023adaac7c3392684aee44f9744407,unknown,page_view,de9eb5f3106683e3aee36af54548f17b,NaN,2022-09-02 00:17:20.943000+03:00,2023-04-26 17:10:50.617251+03:00,340853.0,unknown
26530,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-08 12:05:15.837000+03:00,2022-09-09 10:26:51.558000+03:00,1341.0,unknown
30806,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-09 10:26:51.558000+03:00,2022-09-09 10:31:34.473000+03:00,4.0,3b8294dc079c2882a8fe9b110c8da425
30823,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,page_view,b5ee87931581ac101880ef3043eafd7d,NaN,2022-09-09 10:31:34.473000+03:00,2023-04-26 17:10:50.617251+03:00,330159.0,unknown
44535,0009db22643e3eb23f9098f341738546,unknown,page_view,2ccb2870dd72c5958634a326eee8e50b,NaN,2022-09-14 20:28:06.327000+03:00,2023-04-26 17:10:50.617251+03:00,322362.0,unknown


In [8]:
# добавляем идентификатор
# is_new_session - отсечка, для определения новой сессий
# session_num - номер сессий
df['is_new_session'] = np.where((df["min_diff"]>=120) | (df["transaction_id"].notna()) | \
                                ((df["medium_lead"]!=df["medium"])&(df["medium"]!='unknown')&(df["medium_lead"]!='unknown')) | \
                                ((df["medium"]=='unknown')&(df["medium_lead"]!='unknown')), 1, 0)
df['session_num'] = df.groupby(['user_id'])['is_new_session'].cumsum()
df['session_num'] = np.where(df['is_new_session']==0, df['session_num']+1, df['session_num'])
df.head()

,user_id,medium,event_name,page_location,transaction_id,to_timestamp,to_timestamp_lead,min_diff,medium_lead,is_new_session,session_num
1600,00023adaac7c3392684aee44f9744407,unknown,page_view,de9eb5f3106683e3aee36af54548f17b,NaN,2022-09-02 00:17:20.943000+03:00,2023-04-26 17:10:50.617251+03:00,340853.0,unknown,1,1
26530,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-08 12:05:15.837000+03:00,2022-09-09 10:26:51.558000+03:00,1341.0,unknown,1,1
30806,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-09 10:26:51.558000+03:00,2022-09-09 10:31:34.473000+03:00,4.0,3b8294dc079c2882a8fe9b110c8da425,1,2
30823,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,page_view,b5ee87931581ac101880ef3043eafd7d,NaN,2022-09-09 10:31:34.473000+03:00,2023-04-26 17:10:50.617251+03:00,330159.0,unknown,1,3
44535,0009db22643e3eb23f9098f341738546,unknown,page_view,2ccb2870dd72c5958634a326eee8e50b,NaN,2022-09-14 20:28:06.327000+03:00,2023-04-26 17:10:50.617251+03:00,322362.0,unknown,1,1


In [9]:
# копирование нужных колонок в новый df (в df не скорректрованы маркетинговые каналы)
df_wt_medium  = df[['user_id','medium','event_name','page_location','transaction_id','to_timestamp','to_timestamp_lead','min_diff','medium_lead','session_num']].copy()
df_wt_medium['to_timestamp'] = df_wt_medium['to_timestamp'].dt.tz_localize(None)
df_wt_medium['to_timestamp_lead'] = df_wt_medium['to_timestamp_lead'].dt.tz_localize(None)
df_wt_medium.head()

,user_id,medium,event_name,page_location,transaction_id,to_timestamp,to_timestamp_lead,min_diff,medium_lead,session_num
1600,00023adaac7c3392684aee44f9744407,unknown,page_view,de9eb5f3106683e3aee36af54548f17b,NaN,2022-09-02 00:17:20.943,2023-04-26 17:10:50.617251,340853.0,unknown,1
26530,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-08 12:05:15.837,2022-09-09 10:26:51.558000,1341.0,unknown,1
30806,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-09 10:26:51.558,2022-09-09 10:31:34.473000,4.0,3b8294dc079c2882a8fe9b110c8da425,2
30823,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,page_view,b5ee87931581ac101880ef3043eafd7d,NaN,2022-09-09 10:31:34.473,2023-04-26 17:10:50.617251,330159.0,unknown,3
44535,0009db22643e3eb23f9098f341738546,unknown,page_view,2ccb2870dd72c5958634a326eee8e50b,NaN,2022-09-14 20:28:06.327,2023-04-26 17:10:50.617251,322362.0,unknown,1


In [10]:
# df с маркетинговыми каналами
df_medium = df_wt_medium.copy()
df_medium['rn'] = df_medium.groupby(['user_id','session_num']).cumcount()+1
df_medium = df_medium[df_medium['rn']==1][['user_id', 'medium', 'session_num']]
df_medium.head()

,user_id,medium,session_num
1600,00023adaac7c3392684aee44f9744407,unknown,1
26530,000296445fbdf5da871d39fae90c32de,unknown,1
30806,000296445fbdf5da871d39fae90c32de,unknown,2
30823,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,3
44535,0009db22643e3eb23f9098f341738546,unknown,1


In [11]:
# df с правильными маркетинговыми каналами (заменяем unknown на нужный)
df_bi = df_wt_medium.merge(df_medium, on=['user_id', 'session_num'], how='left')
df_bi['medium'] = np.where(df_bi['medium_x']=='unknown', df_bi['medium_y'], df_bi['medium_x']) 
df_bi = df_bi.drop(columns=['medium_x', 'medium_y'])
df_bi = df_bi[['user_id','medium','event_name','page_location','transaction_id','to_timestamp','to_timestamp_lead','min_diff','medium_lead']]
df_bi.head()

,user_id,medium,event_name,page_location,transaction_id,to_timestamp,to_timestamp_lead,min_diff,medium_lead
0,00023adaac7c3392684aee44f9744407,unknown,page_view,de9eb5f3106683e3aee36af54548f17b,NaN,2022-09-02 00:17:20.943,2023-04-26 17:10:50.617251,340853.0,unknown
1,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-08 12:05:15.837,2022-09-09 10:26:51.558000,1341.0,unknown
2,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-09 10:26:51.558,2022-09-09 10:31:34.473000,4.0,3b8294dc079c2882a8fe9b110c8da425
3,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,page_view,b5ee87931581ac101880ef3043eafd7d,NaN,2022-09-09 10:31:34.473,2023-04-26 17:10:50.617251,330159.0,unknown
4,0009db22643e3eb23f9098f341738546,unknown,page_view,2ccb2870dd72c5958634a326eee8e50b,NaN,2022-09-14 20:28:06.327,2023-04-26 17:10:50.617251,322362.0,unknown


In [12]:
# снова определяем номер сессий, так как маркетинговый канал был скорректирован
df_bi['is_new_session'] = np.where((df_bi["min_diff"]>=120) | (df_bi["transaction_id"].notna()) | \
                                ((df_bi["medium_lead"]!=df_bi["medium"])&(df_bi["medium"]!='unknown')&(df_bi["medium_lead"]!='unknown')) | \
                                ((df_bi["medium"]=='unknown')&(df_bi["medium_lead"]!='unknown')), 1, 0)
df_bi['session_num'] = df_bi.groupby(['user_id'])['is_new_session'].cumsum()
df_bi['session_num'] = np.where(df_bi['is_new_session']==0, df_bi['session_num']+1, df_bi['session_num'])
df_bi = df_bi[['user_id', 'medium', 'event_name', 'page_location', 'transaction_id', 'to_timestamp', 'session_num']]
df_bi.head()

,user_id,medium,event_name,page_location,transaction_id,to_timestamp,session_num
0,00023adaac7c3392684aee44f9744407,unknown,page_view,de9eb5f3106683e3aee36af54548f17b,NaN,2022-09-02 00:17:20.943,1
1,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-08 12:05:15.837,1
2,000296445fbdf5da871d39fae90c32de,unknown,page_view,be4776450a2c2191e6c9c29ecbc747b6,NaN,2022-09-09 10:26:51.558,2
3,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,page_view,b5ee87931581ac101880ef3043eafd7d,NaN,2022-09-09 10:31:34.473,3
4,0009db22643e3eb23f9098f341738546,unknown,page_view,2ccb2870dd72c5958634a326eee8e50b,NaN,2022-09-14 20:28:06.327,1


In [13]:
# экспорт данных в excel файл
df_bi.to_excel('sessions.xlsx')

# Вопрос 2

In [14]:
# копирование df и добавление колонки date
df_bi['date'] = df_bi['to_timestamp'].dt.date
df_conv = df_bi[['user_id', 'medium', 'event_name', 'session_num', 'date']].copy() 
df_conv = df_conv.drop_duplicates() 
df_conv.head()

,user_id,medium,event_name,session_num,date
0,00023adaac7c3392684aee44f9744407,unknown,page_view,1,2022-09-02
1,000296445fbdf5da871d39fae90c32de,unknown,page_view,1,2022-09-08
2,000296445fbdf5da871d39fae90c32de,unknown,page_view,2,2022-09-09
3,000296445fbdf5da871d39fae90c32de,3b8294dc079c2882a8fe9b110c8da425,page_view,3,2022-09-09
4,0009db22643e3eb23f9098f341738546,unknown,page_view,1,2022-09-14


In [15]:
# расчет просмотров и покупок, конверсия по каналам
df_page_view = df_conv[df_conv['event_name']=='page_view'].groupby(['medium'])['event_name'].count().to_frame('views').reset_index()
df_purchase = df_conv[df_conv['event_name']=='purchase'].groupby(['medium'])['event_name'].count().to_frame('purchases').reset_index()
df_result = df_page_view.merge(df_purchase, on=['medium'], how='left') 
df_result['conversion'] = df_result['purchases']/df_result['views']
df_result['conversion'] = round(df_result['conversion']*100, 2) 
df_result['conversion'] = df_result['conversion'].fillna(0) 
df_result = df_result.sort_values(by=['conversion'], ascending=False).reset_index() 
df_result[['medium', 'conversion']]

,medium,conversion
0,338bfbf125696d029d305ff3c3e2f16c,9.68
1,027572bb7194cf20b350c352e71a5084,8.33
2,9446aa3aeddbe34ab778380c8299c555,2.94
3,3b8294dc079c2882a8fe9b110c8da425,2.33
4,unknown,1.72
5,a52f6dfd8cad6dc0e93ad01a508da55f,1.01
6,219338c95a3f5d98e757bf98357345dd,0.61
7,3439029c3f5a75eeb3226ded1b0774d6,0.38
8,e414a106991585be104aec18f5820091,0.15
9,f6950f0c3bb3063f1384e10d8b5de273,0.00
